# IRIS PROCESSING NOTEBOOK for Arctic Template

## ReadMe : 
- This notebook is designed for the spreadsheet of the Arctic Template.
- A few guidelines about the snow pit spreadsheet :
    - IRIS version has to be specified as IRIS# in the IRIS sheet of the pit spreadsheet
    - NoData is filled as 9999 in the spreadsheet when logging the .txt file IRIS provides

<br/><br/>

Author : Paul Billecocq, GRIMP, Université de Sherbooke QC Canada
modif Julien Meloche and Charlotte Crevier

In [1]:
import iris_core as iris
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
import os

# For one specific file

### 1. Input spreadsheet path

In [ ]:
FILENAME = r'Y:\FIELD_WORK\2022\CAMBRIDGEBAY\RAW\20220327\OSSA_SP13_20220327_1100.xlsx' #fill in with your specific filename (it has to be placed in the same folder as the notebook)

#specify iris version from sheet
iris_sheet = "IRIS 3"
iris_version = iris_sheet[:-2] + '_' + iris_sheet[-1:]
print(iris_version)
# Read IRIS data from the IRIS sheet of the file
data = pd.read_excel(FILENAME, sheet_name=iris_sheet)

### 2. Processing

#### IRIS data retrieval, cleaning no data, and average reflectance voltages

In [ ]:
# List cleaning function
def nanify_list(input_list):
    return [np.nan if x == 9999 else x for x in input_list]


# Calibration data retrieval
spectralon = data["Spectralon (%)"].dropna().tolist()
spectralon = np.array(spectralon,dtype=float)
calibration_one = nanify_list(data["Calibration measured IRIS (mV)"].dropna().tolist())
calibration_two = nanify_list(data["Other calibration measured IRIS (mV)"].dropna().tolist())
if np.isnan(np.mean(calibration_one))== True:
    calibration_one = nanify_list(data["Calibration multimeter (mV)"].dropna().tolist())
    calibration_two = nanify_list(data["Other calibration multimeter (mV)"].dropna().tolist())

calibration_list = [calibration_one, calibration_two]
calibration_scans = np.array([calib for calib in calibration_list if len(calib) > 0 ])

# Calibration voltage averaging
calibration = np.nanmean(calibration_scans, axis=0)
calibration = np.array(calibration,dtype=float)

# Measures data retrieval
height = data["Height (cm)"].dropna().tolist()
measures = nanify_list(data['Measured IRIS (mV)'].dropna().tolist())
if np.isnan(np.mean(measures)) == True:
    measures = nanify_list(data["Multimeter (mV)"].dropna().tolist())
measures = np.array(measures)

#### Compute IRIS parameters

In [52]:
# Voltage to reflectance polynomial fit
polynom = iris.calibration_polynom_fit(spectralon, calibration)

In [53]:
# Compute reflectance
reflectance = iris.voltage_to_reflectance(measures, polynom)

In [54]:
# Compute SSA from reflectance
ssa = iris.reflectance_to_ssa(reflectance, iris_version)

In [55]:
# Compute optical radius from SSA
optical_radius = iris.ssa_to_optical_radius(ssa)

In [ ]:
measures_df = pd.DataFrame.from_dict({
    'height': height,
    'voltage': measures,
    'reflectance': reflectance,
    'ssa': ssa,
    'optical radius': optical_radius
})

measures_df

### 3. Write processed data in the spreadsheet

In [11]:
# Spreadsheet initialization
workbook = load_workbook(FILENAME) 
iris_sheet_obj = workbook[iris_sheet]

for i in range(len(measures)):
    iris_sheet_obj.cell(row = i+2, column = 9).value = reflectance[i]
    iris_sheet_obj.cell(row = i+2, column = 10).value = ssa[i]
    iris_sheet_obj.cell(row = i+2, column = 11).value = optical_radius[i]

#For the creation of the file in the processed folder on the serveur  
processed_FILENAME = FILENAME.replace("RAW", "PROCESSED")
path = os.path.join(*processed_FILENAME.split('.')[0].split("\\")[:-1])
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
  # Create a new directory because it does not exist 
  os.makedirs(path)
  print("The new directory is created!")
    
# Write staged changes in the spreadsheet
workbook.save(processed_FILENAME.split('.')[0] + '_IRIS_processed.xlsx')

-------------------------------------------------------------------------------------------------------------------------------

# Batch process folder

In [2]:
#function to batch process a folder
# use with next cell
def batchProcess_SSA(FILENAME):
    
    #specify iris version
    try :
        iris_sheet = "IRIS2"
        iris_version = iris_sheet[:-1] + '_' + iris_sheet[-1:]
        # Read IRIS data from the IRIS sheet of the file
        data = pd.read_excel(FILENAME, sheet_name=iris_sheet)
    except:
        try:
            iris_sheet = "IRIS 2"
            iris_version = iris_sheet[:-2] + '_' + iris_sheet[-1:]
            # Read IRIS data from the IRIS sheet of the file
            data = pd.read_excel(FILENAME, sheet_name=iris_sheet)
        except :
            try:
                iris_sheet = "IRIS 3"
                iris_version = iris_sheet[:-2] + '_' + iris_sheet[-1:]
                # Read IRIS data from the IRIS sheet of the file
                data = pd.read_excel(FILENAME, sheet_name=iris_sheet)
            except:
                iris_sheet = "IRIS3"
                iris_version = iris_sheet[:-1] + '_' + iris_sheet[-1:]
                # Read IRIS data from the IRIS sheet of the file
                data = pd.read_excel(FILENAME, sheet_name=iris_sheet)
    
    # List cleaning function
    def nanify_list(input_list):
        return [np.nan if x == 9999 else x for x in input_list]

    # Calibration data retrieval
    spectralon = data["Spectralon (%)"].dropna().tolist()
    spectralon = np.array(spectralon,dtype=float)
    calibration_one = nanify_list(data["Calibration measured IRIS (mV)"].dropna().tolist())
    calibration_two = nanify_list(data["Other calibration measured IRIS (mV)"].dropna().tolist())
    if np.isnan(np.mean(calibration_one))== True:
        calibration_one = nanify_list(data["Calibration multimeter (mV)"].dropna().tolist())
        calibration_two = nanify_list(data["Other calibration multimeter (mV)"].dropna().tolist())

    #Validate that the iris was done and that all the calibraiton values are in the excel sheet
    if np.isnan(np.mean(calibration_one))== True: #La moyenne sera de np.nan s'il y a un ou plusieur nan dans la liste
        raise NoCalibrationDone
    
    calibration_list = [calibration_one, calibration_two]
    #rtansform list into array
    calibration_scans = np.array([calib for calib in calibration_list if len(calib) > 0 ])
    
    # Calibration voltage averaging from the 2 calibrations
    calibration = np.nanmean(calibration_scans, axis=0)
    calibration = np.array(calibration,dtype=float)

    # Measures data retrieval
    height = data["Height (cm)"].dropna().tolist()
    measures = nanify_list(data['Measured IRIS (mV)'].dropna().tolist())
    if np.isnan(np.mean(measures)) == True:
        measures = nanify_list(data["Multimeter (mV)"].dropna().tolist())
    measures = np.array(measures)
    polynom = iris.calibration_polynom_fit(spectralon, calibration)
    reflectance = iris.voltage_to_reflectance(measures, polynom)
    ssa = iris.reflectance_to_ssa(reflectance, iris_version)
    optical_radius = iris.ssa_to_optical_radius(ssa)
    try: 
        measures_df = pd.DataFrame.from_dict({
        'height': height,
        'voltage': measures,
        'reflectance': reflectance,
        'ssa': ssa,
        'optical radius': optical_radius
        })
    except :
        raise ListNotSameLenght
        
    # Spreadsheet initialization
    workbook = load_workbook(FILENAME) 
    iris_sheet_obj = workbook[iris_sheet]

    for i in range(len(measures)):
        iris_sheet_obj.cell(row = i+2, column = 9).value = reflectance[i]
        iris_sheet_obj.cell(row = i+2, column = 10).value = ssa[i]
        iris_sheet_obj.cell(row = i+2, column = 11).value = optical_radius[i]
    
    #For the creation of the file in the processed folder on the serveur  
    processed_FILENAME = FILENAME.replace("RAW", "PROCESSED")
    path = os.path.join(*processed_FILENAME.split('.')[0].split("\\")[:-1])
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
      # Create a new directory because it does not exist 
      os.makedirs(path)
      print("The new directory is created!")

    # Write staged changes in the spreadsheet
    workbook.save(processed_FILENAME.split('.')[0] + '_IRIS_processed.xlsx')
    
class NoCalibrationDone(Exception):
    pass
class ListNotSameLenght(Exception):
    pass

In [3]:
#Liste des dates à traiter
#Liste des dates pour lesquelles nous avons fait des SP et qui se trouve dans le dossier de terrain
list_of_date = ['20220326', '20220327', '20220328', '20220329', '20220401', '20220402', '20220403', '20220404', '20220405', 
                '20220406', '20220409', '20220410', '20220411', '20220412', '20220414', '20220415', '20220416', '20220417', 
               '20220418', '20220419', '20220420', '20220421', '20220422', '20220423', '20220425', '20220426', '20220427']

#Chemin vers le dossier sur le serveur
main_folder = r'Y:\FIELD_WORK\2022\CAMBRIDGEBAY\RAW'


In [ ]:
import os
#Create a list of the not working file 
error_file = []
missing9999_file = []
noCalibration_file = []
file_done = []
file_total = []
#specify data folder for batch process
for date in list_of_date:
    
    data_folder = os.path.join(main_folder, date)
    #data_folder = "D:CB-2022-radiometer/snowpit_SSAtoProcess"
    print('Starting : ', data_folder)
    for file in os.listdir(data_folder):
        if ".xlsx" in file:
            file_total.append(file)
            try:
                batchProcess_SSA(os.path.join(data_folder,file))
            except NoCalibrationDone: #S'il manque 1 ou plus valeurs de iris dans la calibration
                noCalibration_file.append(file)
            except ListNotSameLenght: # Si les listes de hauteur de neige et de valeur de IRIS ne sont pas de la même longueur
                missing9999_file.append(file)
            except :
                error_file.append(file) #Autre erreur avec le fichier
            else :
                file_done.append(file)
    print('Done with : ', data_folder)

In [ ]:
#get the list of the file that the process didn't work (could be exported in .txt for futur reference)
error_file

In [ ]:
#get the list of the file that the process didn't work because of the calibration (could be exported in .txt for futur reference)
noCalibration_file

In [ ]:
#get the list of the file that the process didn't work because number of measurments doesn't fit the height (could be exported in .txt for futur reference)
missing9999_file